# Fastcat Demo: Varian kV OBI

In [60]:
%pylab widget
%load_ext autoreload
%autoreload 2

[2021-06-21 14:57:13,299] {pyplot.py:290} DEBUG - Loaded backend module://ipympl.backend_nbagg version unknown.
Populating the interactive namespace from numpy and matplotlib
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [61]:
import fastcat as fc

# Initialize the spectrum

In [62]:
s = fc.calculate_spectrum(100,14,3,50,monitor=None)

In [63]:
s.attenuate(0.4,fc.get_mu(z=13)) # Aluminum inherent filtration
s.attenuate(0.089,fc.get_mu(z=22)) # Titanium beam hardening filter

In [64]:
plt.figure()
s.get_plot(plt.subplot())

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Initialize the detector

In [65]:
det = fc.Detector(s,'CZT-342-micrometer')
det.add_focal_spot(0.6)

plt.figure()
det.get_plot(plt.subplot())

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Initialize the phantom

In [88]:
phantom = fc.Catphan_404_Devon(hi_res=False)
phantom.PCD = False

[2021-06-22 12:47:08,849] {phantoms.py:878} INFO - Phantom is low resolution


# Run the simulation

In [89]:
angles = np.linspace(0,np.pi*2,180,endpoint=False)
phantom.return_projs(det,s,angles,ASG=False)

[2021-06-22 12:47:09,884] {simulate.py:465} INFO - Running Simulations
[2021-06-22 12:47:09,884] {simulate.py:469} INFO -     0.6 mm focal spot added
[2021-06-22 12:47:09,885] {simulate.py:480} INFO -     Simulating 10 keV
[2021-06-22 12:47:12,065] {simulate.py:480} INFO -     Simulating 20 keV
[2021-06-22 12:47:14,269] {simulate.py:480} INFO -     Simulating 30 keV
[2021-06-22 12:47:16,755] {simulate.py:480} INFO -     Simulating 40 keV
[2021-06-22 12:47:19,447] {simulate.py:480} INFO -     Simulating 50 keV
[2021-06-22 12:47:22,067] {simulate.py:480} INFO -     Simulating 60 keV
[2021-06-22 12:47:24,679] {simulate.py:480} INFO -     Simulating 70 keV
[2021-06-22 12:47:27,362] {simulate.py:480} INFO -     Simulating 80 keV
[2021-06-22 12:47:30,107] {simulate.py:480} INFO -     Simulating 90 keV
[2021-06-22 12:47:32,777] {simulate.py:480} INFO -     Simulating 100 keV
[2021-06-22 12:47:35,265] {simulate.py:612} INFO - Weighting simulations
[2021-06-22 12:47:35,265] {simulate.py:686} IN

In [77]:
class IndexTracker(object):
    def __init__(self, ax, X):
        self.ax = ax
        ax.set_title('use scroll wheel to navigate images')

        self.X = X
        rows, cols, self.slices = X.shape
        self.ind = 10 #self.slices//2

        self.im = ax.imshow(self.X[:, :, self.ind])
        self.update()

    def onscroll(self, event):
        print("%s %s" % (event.button, event.step))
        if event.button == 'up':
            self.ind = (self.ind + 1) % self.slices
        else:
            self.ind = (self.ind - 1) % self.slices
        self.update()

    def update(self):
        self.im.set_data(self.X[:, :, self.ind])
        ax.set_ylabel('slice %s' % self.ind)
        self.im.axes.figure.canvas.draw()

fig, ax = plt.subplots(1, 1)

X = np.array(phantom.proj.T)

tracker = IndexTracker(ax, X)

fig.canvas.mpl_connect('scroll_event', tracker.onscroll)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [82]:
plt.figure()
plt.imshow(phantom.proj[90],cmap='gray')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Reconstruct

In [90]:
phantom.reconstruct('FDK',filt='ram_lak')

plt.figure()
plt.imshow(phantom.img[5] - im_pcd,cmap='gray')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [87]:
phantom.reconstruct('FDK',filt='ram_lak')

plt.figure()
plt.imshow(phantom.img[5],cmap='gray')
im_pcd = phantom.img[5]

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [53]:
phantom.geomet.sVoxel = np.array([100,90,90])

phantom.geomet.dVoxel = phantom.geomet.sVoxel / phantom.geomet.nVoxel

In [70]:
print(phantom.geomet)

TIGRE parameters
-----
Geometry parameters
Distance from source to detector (DSD) = 588 mm
Distance from source to origin (DSO)= 322 mm
-----
Detector parameters
Number of pixels (nDetector) = [ 64 576]
Size of each pixel (dDetector) = [0.33 0.33] mm
Total size of the detector (sDetector) = [ 21.12 190.08] mm
-----
Image parameters
Number of voxels (nVoxel) = [ 10 512 512]
Total size of the image (sVoxel) = [ 50 100 100] mm
Size of each voxel (dVoxel) = [5.        0.1953125 0.1953125] mm
-----
Offset correction parameters
Offset of image from origin (offOrigin) = [0 0 0] mm
Offset of detector (offDetector) = [0 0] mm
-----
Auxillary parameters
Samples per pixel of forward projection (accuracy) = 0.5
-----
Rotation of the Detector (rotDetector) = [0 0 0] rad
